# Curl example
```
curl http://localhost:1234/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "meta-llama-3.1-8b-instruct",
    "messages": [
      { "role": "system", "content": "Always answer in rhymes. Today is Thursday" },
      { "role": "user", "content": "What day is it today?" }
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": false
}'
```

In [63]:
import requests
import json
import textwrap
import pandas as pd
from datetime import datetime

max_width = 80

url = "http://localhost:1234/v1/chat/completions"

headers = {
    'Content-Type': 'application/json'
}

# initial state of data
data = {
        "model": "meta-llama-3.1-8b-instruct",
        "messages": [
            { "role": "system", "content": "You will always make a description of a scene or action happening with in it" }
        ],
        "temperature": 0.7,
        "max_tokens": -1,
        "stream": False
    }

# import spreadsheet data
df = pd.read_csv('llm_game_data/llm_game_data/Sheet 1-locations.csv')

In [64]:
# write function to gather initial user preferences (setting, character types, conflict)
def game_setting():
    setting = input("What type of setting would you like to interact with (desert, rainforest, etc). Be as specific as you'd like.")
    people_animals = input("What type of people/animals would be in this setting? Describe a few different personalities.")
    conflict_goal = input("Is there any kind of conflict or goal you'd like to see in this environment?")

    # append info to data
    setting = { "role": "system", "content": f"The scene looks like {setting}" }
    data['messages'].append(setting)
    people_animals = { "role": "system", "content": f"The people/animals in the setting are {people_animals}" }
    data['messages'].append(people_animals)
    conflict_goal = { "role": "system", "content": f"The user wishes to resolve or achieve {conflict_goal}" }
    data['messages'].append(conflict_goal)

In [65]:
# Export data
def save_data():
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
    filename = f"saves/output_{current_datetime}.json"
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

In [66]:
# create a function that queries API, prints a descripton and then asks for user input
def game_loop(data):
    # gather input
    headers = {
        'Content-Type': 'application/json'
    }

    # prompt for initial settings from user
    create_world = input('Do you wish to create a specific game world? (y/n)')
    if create_world.lower() == 'y':
        game_setting()

    # initial location
    x = 0
    y = 0
    location = df[(df['x'] == x) & (df['y'] == y)]['description'].values[0]

    user_prompt = ''
    while user_prompt.lower() != 'exit':
        user_prompt = input("What would you like to do (use n, s, e, w for movement)?")

        # pull location from df
        if user_prompt.lower() in ['n', 's', 'e', 'w']:
            # update x/y values
            if user_prompt.lower() == 'n':
                y+=1
                user_prompt = "I move north"
            elif user_prompt.lower() == 's':
                y-=1
                user_prompt = "I move south"
            elif user_prompt.lower() == 'e':
                x+=1
                user_prompt = "I move east"
            elif user_prompt.lower() == 'w':
                x-=1
                user_prompt = "I move west"
        
        # TODO: this handles extra prompt when exit is typed (optimize later)
        if user_prompt.lower() != 'exit':
            # add location to system messages only if location hasn't changed from previous loop
            old_location = location
            location = df[(df['x'] == x) & (df['y'] == y)]['description'].values[0]
            if old_location == location:
                data['messages'].append({ "role": "system", "content":f"Present at the current location is {location}" })
            print(f'\n({x}, {y}): {location}')

            # add user's response to data and print out
            data['messages'].append({ "role": "user", "content": user_prompt })
            print(f'\nUser: {user_prompt}\n')

            response = requests.post(url, headers=headers, json=data)

            # print response
            response_json = response.json()
            message = response_json["choices"][0]["message"]["content"]
            wrapped_text = textwrap.wrap(message, width=max_width)
            print("\n".join(wrapped_text))

            # add assistant response to data
            data['messages'].append(response_json["choices"][0]["message"])
        else:
            # save data
            save_data()

In [67]:
game_loop(data)


(0, 0): The user will need help to navigate this environment.


User: look around

As you look around, the dense foliage of the tropical rainforest surrounds you.
The air is thick with the sounds of chirping birds and the rustling of leaves.
Brightly colored birds flit about, their vibrant plumage flashing in the dappled
sunlight filtering through the canopy above.  To your left, a brilliant blue-
and-yellow macaw perches on a branch, its long tail feathers swishing lazily
back and forth. Near the base of the tree, a group of locals, dressed in
colorful traditional clothing, are gathered around a clearing. They seem to be
discussing something among themselves, their voices hushed but animated.  To
your right, a steep hill rises up, covered in thick vegetation. The sound of
rushing water echoes from somewhere within the dense underbrush. Suddenly, a
loud, jarring noise pierces the air, making you jump. It sounds like a car
backfiring or a chainsaw being turned on.  The locals notice yo